In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from feature_engine.encoding import RareLabelEncoder

In [2]:
df_resort = pd.read_csv('../Dataset/h1.csv')#Resort
df_city = pd.read_csv('../Dataset/h2.csv') #City

print("Shape of h1:", df_resort.shape)
print("Shape of h2:", df_city.shape)

df_resort['HotelType'] = 'Resort'
df_city['HotelType'] = 'City'

df = pd.concat([df_resort, df_city])

df.info()

Shape of h1: (40060, 31)
Shape of h2: (79330, 31)
<class 'pandas.core.frame.DataFrame'>
Index: 119390 entries, 0 to 79329
Data columns (total 32 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   IsCanceled                   119390 non-null  int64  
 1   LeadTime                     119390 non-null  int64  
 2   ArrivalDateYear              119390 non-null  int64  
 3   ArrivalDateMonth             119390 non-null  object 
 4   ArrivalDateWeekNumber        119390 non-null  int64  
 5   ArrivalDateDayOfMonth        119390 non-null  int64  
 6   StaysInWeekendNights         119390 non-null  int64  
 7   StaysInWeekNights            119390 non-null  int64  
 8   Adults                       119390 non-null  int64  
 9   Children                     119386 non-null  float64
 10  Babies                       119390 non-null  int64  
 11  Meal                         119390 non-null  object 
 12  Country       

### Data Cleaning

First of all, we have to individuate columns which contain Nan Values. It's possible that some columns contais string values as "NULL" or "Undefined"

In [3]:
na_percent = df.isnull().mean() * 100
print(na_percent[na_percent > 0])

Children    0.003350
Country     0.408744
dtype: float64


For the Children column we simple replace NaN values with 0 values.
For the country variable we replace the NaN values with the Unknown string, then going to better explore the relationship between the country and the deletions, so as to prevent data leakage as suggested in the paper.



In [4]:
null_percent = pd.Series(dtype=float)

for col in df.columns:
    col_values = df[col].astype(str).str.strip().str.upper()
    count_null = (col_values == "NULL").sum()
    percent_null = (count_null / len(df)) * 100
    if percent_null > 0:
        null_percent[col] = percent_null

print("Percentage of 'NULL' values:")
print(null_percent)

Percentage of 'NULL' values:
Agent      13.686238
Company    94.306893
dtype: float64


As suggested in the dataset paper, some categorical variables may have NULL values, which in the case of Agent and Company means that those bookings were not made by agencies and the payment for the booking is not handled by them.  
If not already done, we will replace all NULL values in Agent and Company with the integer 0.

In [5]:
company_zero_count = df['Company'].astype(str).str.strip() == '0'
agent_zero_count = df['Agent'].astype(str).str.strip() == '0'

print(f"Valori 0 in 'Company': {company_zero_count.sum()}")
print(f"Valori 0 in 'Agent': {agent_zero_count.sum()}")



Valori 0 in 'Company': 0
Valori 0 in 'Agent': 0


0 has not been used in either case, so we can replace NULL values with 0.

For the Meal variable, it is possible to replace Undefined with SC, which indicates the service without included meals.
As for the other two variables, these Undefined values do not have a specific meaning, so given their very small number, it is also possible to consider removing the rows that contain these values.

In [6]:
undefined_percent = pd.Series(dtype=float)

for col in df.columns:
    col_values = df[col].astype(str).str.strip()
    count_undefined = (col_values == "Undefined").sum()
    percent_undefined = (count_undefined / len(df)) * 100
    if percent_undefined > 0:
        undefined_percent[col] = percent_undefined

print("Percentage of 'Undefined' values: ")
print(undefined_percent)


Percentage of 'Undefined' values: 
Meal                   0.979144
MarketSegment          0.001675
DistributionChannel    0.004188
dtype: float64


Dataset Cleaning

In [7]:
# 1. Replace NaN in 'Children' with 0
df['Children'] = df['Children'].fillna(0)

# 2. Replace NaN in 'Country' with 'Unknown'
df['Country'] = df['Country'].fillna('Unknown')

# 3. Replace 'NULL' with 0 in the 'Agent' and 'Company' columns
df['Agent'] = df['Agent'].astype(str).str.strip().replace('NULL', 0)
df['Company'] = df['Company'].astype(str).str.strip().replace('NULL', 0)

# 4. Replace 'Undefined' with 'SC' in the 'Meal' column
df['Meal'] = df['Meal'].astype(str).str.strip().replace('Undefined', 'SC')

# 5. Remove rows with 'Undefined' in 'MarketSegment' or 'DistributionChannel'
df = df[
    (df['MarketSegment'].astype(str).str.strip() != 'Undefined') &
    (df['DistributionChannel'].astype(str).str.strip() != 'Undefined')
]

We then check if there are any bookings that do not include any people.

In [8]:
percent_all_zero = (((df['Adults'] == 0) & (df['Children'] == 0) & (df['Babies'] == 0)).sum()/ len(df)) * 100

print(f"Percentage of rows with 'Adults', 'Children' e 'Babies' = 0: {percent_all_zero:.2f}%")


Percentage of rows with 'Adults', 'Children' e 'Babies' = 0: 0.15%


Delete these rows

In [9]:
df = df[~((df['Adults'] == 0) & (df['Children'] == 0) & (df['Babies'] == 0))]

In [10]:
df['Company'].nunique(), df['Agent'].nunique()

(349, 334)

In [11]:
pd.set_option('display.max_rows', None)
df['Company'].value_counts(normalize=True).sort_values(ascending=False)



Company
0      0.943224
40     0.007751
223    0.006577
67     0.002240
45     0.002089
153    0.001787
174    0.001233
219    0.001183
281    0.001158
154    0.001116
405    0.000998
233    0.000948
51     0.000831
94     0.000730
47     0.000596
135    0.000554
169    0.000537
242    0.000520
331    0.000512
348    0.000495
498    0.000487
110    0.000436
38     0.000419
20     0.000419
342    0.000403
280    0.000403
91     0.000403
62     0.000394
197    0.000394
68     0.000386
218    0.000361
270    0.000361
202    0.000319
195    0.000319
148    0.000310
9      0.000310
113    0.000302
307    0.000302
204    0.000285
308    0.000277
238    0.000277
269    0.000277
86     0.000268
385    0.000252
72     0.000252
343    0.000243
365    0.000243
43     0.000243
221    0.000227
178    0.000227
144    0.000227
46     0.000218
337    0.000210
418    0.000210
366    0.000201
179    0.000201
424    0.000201
227    0.000201
81     0.000193
507    0.000193
477    0.000193
407    0.000185


In [12]:
df['Agent'].value_counts(normalize=True).sort_values(ascending=False)

Agent
9      0.267782
0      0.136546
240    0.116790
1      0.060291
14     0.030469
7      0.029630
6      0.027600
250    0.024076
241    0.014437
28     0.013900
8      0.012701
3      0.011208
37     0.010302
19     0.008859
40     0.008716
314    0.007777
21     0.007324
229    0.006569
242    0.006543
83     0.005839
29     0.005721
171    0.005092
12     0.004840
85     0.004639
20     0.004530
96     0.004505
243    0.004312
30     0.004060
134    0.004043
298    0.003960
27     0.003767
15     0.003364
26     0.003364
11     0.003263
22     0.003196
56     0.003146
273    0.002928
177    0.002911
58     0.002810
86     0.002810
5      0.002768
119    0.002550
196    0.002525
34     0.002466
44     0.002450
138    0.002408
315    0.002382
38     0.002290
10     0.002181
236    0.002072
16     0.002064
17     0.002013
115    0.001888
251    0.001846
68     0.001770
42     0.001770
191    0.001661
175    0.001636
195    0.001619
154    0.001619
156    0.001594
168    0.001535
15

As we can see, both variables have very high cardinality, but most of the values are infrequent within the dataset. Therefore, it is advisable to perform rare encoding with a threshold set at 2%.

In [13]:
rare_encoder = RareLabelEncoder(
    tol=0.02,                    
    variables=['Agent', 'Company'],  
    replace_with='Other'        
)


df = rare_encoder.fit_transform(df)

In [14]:
print("Company cardinality:", df['Company'].nunique())
print("Agent cardinality:", df['Agent'].nunique())

Company cardinality: 2
Agent cardinality: 9


In [15]:
df['Company'].value_counts(normalize=True).sort_values(ascending=False)

Company
0        0.943224
Other    0.056776
Name: proportion, dtype: float64

In [16]:
df['Agent'].value_counts(normalize=True).sort_values(ascending=False)

Agent
Other    0.306816
9        0.267782
0        0.136546
240      0.116790
1        0.060291
14       0.030469
7        0.029630
6        0.027600
250      0.024076
Name: proportion, dtype: float64

## Feature Selection

In [17]:
country_counts = df['Country'].value_counts(dropna=False)
country_percent = (country_counts / country_counts.sum()) * 100
print(country_percent.to_string())

Country
PRT        40.667757
GBR        10.167359
FRA         8.725305
ESP         7.180907
DEU         6.111321
ITA         3.155069
IRL         2.830418
BEL         1.964683
BRA         1.864016
NLD         1.764188
USA         1.755799
CHE         1.447926
CN          1.072942
AUT         1.059519
SWE         0.855669
CHN         0.838052
POL         0.770102
ISR         0.561218
RUS         0.528501
NOR         0.509207
ROU         0.419445
Unknown     0.400990
FIN         0.374145
DNK         0.364918
AUS         0.357368
AGO         0.303679
LUX         0.239923
MAR         0.217273
TUR         0.208045
HUN         0.192945
ARG         0.179523
JPN         0.165262
CZE         0.143450
IND         0.126673
KOR         0.111573
GRC         0.107378
DZA         0.086406
SRB         0.084728
HRV         0.083889
MEX         0.071306
EST         0.069628
IRN         0.068789
LTU         0.067950
ZAF         0.067111
BGR         0.062917
NZL         0.062078
COL         0.059561
UKR  

As stated in the paper, the country column could be a source of data leakage. This is because the value Portugal is automatically assigned in most bookings, and this value is only corrected when a check-in is actually made. This means that the majority of canceled bookings have Portugal as the value in the Country column.

In [18]:

country_counts_not_canceled = df[df['IsCanceled'] == 0]['Country'].value_counts(dropna=False)
country_percent_not_canceled = (country_counts_not_canceled / country_counts_not_canceled.sum()) * 100

country_counts_canceled = df[df['IsCanceled'] == 1]['Country'].value_counts(dropna=False)
country_percent_canceled = (country_counts_canceled / country_counts_canceled.sum()) * 100

country_diff = country_percent_canceled.subtract(country_percent_not_canceled, fill_value=0)

country_diff_abs = country_diff.abs()

top10_countries = country_diff_abs.sort_values(ascending=False).head(10).index

print("Top 10 countries with the largest percentage difference:")
print(country_diff[top10_countries].sort_values(ascending=False).to_string())

Top 10 countries with the largest percentage difference:
Country
PRT    34.264487
AUT    -0.856729
USA    -0.988771
NLD    -1.412030
BEL    -1.417815
IRL    -1.506316
ESP    -3.583635
DEU    -5.332287
FRA    -6.915364
GBR    -7.340809


As we can see, canceled bookings with Portugal as the Country value are significantly more numerous than confirmed bookings, supporting what we previously stated. For this reason, the Country column should be removed from the dataset.

In [19]:
df = df.drop(columns=['Country'])

To prevent data leakage, it is also necessary to remove the feature ReservedRoomType. Let’s examine the mismatch between ReservedRoomType and AssignedRoomType based on whether the booking was canceled or not.

In [ ]:
not_canceled = df[df['IsCanceled'] == 0]
total_not_canceled = len(not_canceled)
mismatch_not_canceled = (not_canceled['ReservedRoomType'] != not_canceled['AssignedRoomType']).sum()
percent_not_canceled = (mismatch_not_canceled / total_not_canceled) * 100

canceled = df[df['IsCanceled'] == 1]
total_canceled = len(canceled)
mismatch_canceled = (canceled['ReservedRoomType'] != canceled['AssignedRoomType']).sum()
percent_canceled = (mismatch_canceled / total_canceled) * 100

print("Not Canceled Bookings:")
print(f"Totale: {total_not_canceled}, Mismatch: {mismatch_not_canceled}, Percentuale: {percent_not_canceled:.2f}%\n")

print("Canceled Bookings:")
print(f"Totale: {total_canceled}, Mismatch: {mismatch_canceled}, Percentuale: {percent_canceled:.2f}%")


Not Canceled Bookings:
Totale: 75010, Mismatch: 13995, Percentuale: 18.66%

Canceled Bookings:
Totale: 44195, Mismatch: 801, Percentuale: 1.81%


As we can see, in the case of canceled bookings, the mismatch is significantly different compared to non-canceled bookings, and this could lead to data leakage, since we are providing the model with information that could likely change after check-in. The solution is therefore to remove these two features.

In [21]:
df = df.drop(columns=['AssignedRoomType'])

Obviously, the columns ReservationStatus and ReservationStatusDate must also be removed, as they definitely cause data leakage.

In [22]:
df = df.drop(columns=['ReservationStatus','ReservationStatusDate','RequiredCarParkingSpaces'])

## Feature Engineering

ADR represents the average daily rate paid by the customer, which depends on various factors such as room type, season, or booking channel. The problem is that prices can be highly variable due to inflation and special events, and by removing temporal features from the dataset to avoid data leakage, the ADR feature loses its meaning. The solution is to introduce a normalized variable that compares the price of a booking with similar bookings, calculated as follows:

\begin{equation} ADRThirdQuartileDeviation= \frac{ADR}{Q3 ADR} \end{equation}

where Q3 ADR is calculated for each booking group based on:

1) Same distribution channel
2) Same Reserved Room Type
3) Same expected week/year of arrival


In [23]:
save_path = '../Dataset/df_cleaned.csv'
df.to_csv(save_path, index=False)